#### Import all libraries

In [1]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy as np
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm, getTestCase
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import logging as logger
import time
import json
import os
import logging.config
import pandas as pd
import gc
import math

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#Read the entire Benthiemer sandstone
global image_1, label_1
#change the path to the correct directory before running the code
# please note: the dimensions of the image is 800 x 800 x 800
image_1 = sitk.ReadImage('D:\\Ankita PhD\\pyradiomics\\data\\data3.nrrd')

#### Extraction Settings

In [4]:
settings = {}
settings['label'] = 255
settings['binWidth'] = 1
settings['padDistance'] = 0
settings['distances'] = [1]

#### Log messages

In [5]:
%%time
# the size of the mask should be the same as that of the image
mask_x = 600
mask_y = 600
mask_z = 600

v1 = []
v2 = []
v3 = []
v4 = []
v5 = []
v6 = []
v7 = []
v8 = []
v9 = []
v10 = []
v11 = []
v12 = []
v13 = []
v14 = []
v15 = []
v16 = []

# here the window_size refers to the region of mask that will be used to segment the ROI from the image. The window
# sizes increase with every iteration
u_window_size = [150] #10, 20, 30, 40, 50, 60, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800]
global window_size, total_iter, expect_iter, i_loc, j_loc, z_loc, bb, correctedMask, croppedImage, croppedMask, glcm_window
for window_size in u_window_size:  
    
    i_loc = int((mask_x - window_size)/2)
    j_loc = int((mask_y - window_size)/2)
    k_loc = int((mask_z - window_size)/2)
    
    # Generate the mask
    full_mask = np.zeros((mask_x, mask_y, mask_z), dtype = np.uint8)
    full_mask[i_loc:i_loc+window_size,j_loc:j_loc+window_size,k_loc:k_loc+window_size] = 255
    #print("Window Location", str(i_loc), str(i_loc+window_size), str(j_loc), str(j_loc+window_size), str(k_loc), str(k_loc+window_size))
    label_1 = sitk.GetImageFromArray(full_mask, isVector=False)
    del full_mask

    bb, correctedMask = imageoperations.checkMask(image_1, label_1, label=255)
    if correctedMask is not None:
        label_1 = correctedMask
    croppedImage, croppedMask = imageoperations.cropToTumorMask(image_1, label_1, bb)
    del correctedMask, label_1

    #calculate secondorder stats
    glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)
    glszmFeatures.enableAllFeatures()
    print('Calculating GLSZM features...',)
    result = glszmFeatures.execute()
    print('done')
    
    #print('Calculated GLSZM features: ')
    #for key, value in six.iteritems(result):
     #   print('\t', key, ':', value)
     
    print("Completed Domain Calculations for Window ", str(window_size))

Calculating GLSZM features...
done
Completed Domain Calculations for Window  150
Wall time: 4.77 s


In [20]:
Ng = glszmFeatures.coefficients['Ng']
Ng

251

In [6]:
im_exp = sitk.GetArrayFromImage(croppedImage)
im_exp.shape

(150, 150, 150)

In [14]:
np.max(im_exp)

255

In [17]:
np.min(im_exp)

5

In [7]:
glszm_3 = glszmFeatures.P_glszm
glszm_3

array([[[2., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [6., 0., 0., ..., 0., 0., 0.],
        [3., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 1., 1., 1.]]])

In [8]:
glszm_3.shape

(1, 250, 37)

In [9]:
type(glszm_3)

numpy.ndarray

In [10]:
glszm = glszm_3[0, 0:250, 0:37]
glszm.shape

(250, 37)

In [12]:
#Original GLSZM
pd.DataFrame(glszm).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data3_glszm_org_150.csv")

In [16]:
#The graylevels in ROI after discretization
graylevels = glszmFeatures.coefficients['grayLevels']
graylevels

array([  1,   2,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
       119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131,
       132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144,
       145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 18

In [30]:
np.unique(im_exp)

array([  5,   6,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,
        19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,
        32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,
        45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,
        58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,
        71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,
        97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109,
       110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
       123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135,
       136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148,
       149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161,
       162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174,
       175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 18

In [ ]:
# sum of all the size zones for all the graylevels of a particular size zones
# for every column of GLSZM, add all the rows
pss = glszmFeatures.coefficients['ps']
pss

In [ ]:
pss.shape

In [ ]:
type(pss)

In [ ]:
#Np is the number of voxels in the image
Npp = glszmFeatures.coefficients['Np']
Npp

In [18]:
#The total number of size zones in the GLSZM (add all the elements of P_glszm)
Nzz = glszmFeatures.coefficients['Nz']
Nzz
type(Nzz)

array([447117.])

numpy.ndarray

In [ ]:
#Number of all possible discreet intensity values in the image: This incluedes  NgVector = range(1, Ng + 1) i.e.
#all the missing grayvalues as well. Later the missing grayvalues are excluded. 
Ngg = glszmFeatures.coefficients['Ng']
Ngg

In [ ]:
#sum all the size zones for a particular graylevel
#for every row of GLSZM add the columns
pgg = glszmFeatures.coefficients['pg']
pgg

In [ ]:
pgg.shape

In [ ]:
pg_org = np.sum(glszm, 1)
pg_reshape = np.reshape(pg_org, (1,201))
pg_reshape.shape

In [ ]:
pg_org
pg_org.shape

In [13]:
ivv = glszmFeatures.coefficients['ivector']
ivv

array([  1.,   2.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,
        13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,
        24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,
        35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,
        46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,
        57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,
        68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,
        79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,
        90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.,
       101., 102., 103., 104., 105., 106., 107., 108., 109., 110., 111.,
       112., 113., 114., 115., 116., 117., 118., 119., 120., 121., 122.,
       123., 124., 125., 126., 127., 128., 129., 130., 131., 132., 133.,
       134., 135., 136., 137., 138., 139., 140., 141., 142., 143., 144.,
       145., 146., 147., 148., 149., 150., 151., 15

In [ ]:
ivv.shape

In [15]:
jvv = glszmFeatures.coefficients['jvector']
jvv

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  31.,  33.,  34.,  39.,
        51.,  71.,  78., 134.])

In [ ]:
jvv.shape

In [ ]:
#Original jvector
pd.DataFrame(jvv).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data16_jvv_org.csv")

In [1]:
#Original ivector
pd.DataFrame(ivv).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data16_ivv_org.csv")

NameError: name 'pd' is not defined

In [ ]:
ivv.size

In [ ]:
ivv[ivv.size - 1]

In [ ]:
#Remove the mineral or saturated phase ROW if it exists
if ivv[ivv.size - 1] == 256:
    print('Mineral phase is present because the graylevel of', ivv[ivv.size-1], 'exists.')
    mineralGrayLevels_idx = ivv.size-1
    glszm_3_nm = np.delete(glszm_3, mineralGrayLevels_idx, 1)
    glszm_3_nm.shape
    s_nm = glszm_3_nm.shape

In [ ]:
glszm_nm_row = glszm_3_nm[0, 0:200, 0:426]
pd.DataFrame(glszm_nm_row).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data16_glszm_nm_row_256.csv")

In [ ]:
#new ivector without mineral phase row = 256
ivec_nm_row = ivv[0:ivv.size-1]  # shape (Ng,)
ivec_nm_row

In [ ]:
#new ivector after removing mineral row
pd.DataFrame(ivec_nm_row).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data16_ivec_nm_row.csv")

In [ ]:
# Delete columns that specify zone sizes not present in the ROI
glszm_2d_nm = glszm_3_nm[0, 0:200, 0:426]
ps_new = np.sum(glszm_2d_nm, 0)
type(ps_new)

In [ ]:
ps_new.shape

In [ ]:
ps_reshape = np.reshape(ps_new, (1,426))
ps_reshape.shape

In [ ]:
emptyZoneSizes = np.where(np.sum(ps_reshape, 0) == 0)
emptyZoneSizes

In [ ]:
glszm_nmm = np.delete(glszm_3_nm, emptyZoneSizes, 2)
glszm_nmm.shape

In [ ]:
glszm_nmm_new = glszm_nmm[0, 0:200, 0:257]
pd.DataFrame(glszm_nmm_new).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data16_glszm_nm_row_256_empty_size_zones.csv")

In [ ]:
jv_cmplete = np.delete(jvv, emptyZoneSizes)

In [ ]:
jv_cmplete.shape

In [ ]:
pd.DataFrame(jv_cmplete).to_csv("D:\\Ankita PhD\\pyradiomics\\data\\data16_jvv_complete.csv")

In [ ]:
ps_new_afterreshape = np.delete(ps_reshape, emptyZoneSizes, 1)
ps_new_afterreshape

In [ ]:
# Get the number of zones in this GLSZM
Nz_new = np.sum(glszm_nmm, (1, 2))  # shape (Nvox,)
Nz_new[Nz_new == 0] = 1  # set sum to numpy.spacing(1) if sum is 0?
Nz_new

In [ ]:
# Get the number of voxels represented by this GLSZM: Multiply the zones by their size and sum them
Np_new = np.sum(ps_new_afterreshape * jv_cmplete[None, :], 1)  # shape (Nvox, )
Np_new[Np_new == 0] = 1

In [ ]:
hilae = np.sum(glszm_nmm * (ivec_nm_row[None, :, None] ** 2) * (jv_cmplete[None, None, :] ** 2), (1, 2)) / Nz_new
hilae

In [23]:
NgVector = range(1, Ng + 1)  # All possible gray values
 # Gray values present in ROI
emptyGrayLevels = np.array(list(set(NgVector) - set(graylevels)))

In [24]:
NgVector

range(1, 252)

In [25]:
emptyGrayLevels

array([3])

In [27]:
glszm_3_new = np.delete(glszm_3, emptyGrayLevels - 1, 1)

In [28]:
glszm_3_new

array([[[2., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [4., 0., 0., ..., 0., 0., 0.],
        ...,
        [6., 0., 0., ..., 0., 0., 0.],
        [3., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 1., 1., 1.]]])